# Основы глубокого обучения 

1) Линейный softmax классификатор  
2) Обучение линейного классфикатора  
3) Многослойный персептрон  
4) Рекуррентные сети.

### 1) Линейный softmax классификатор

Рассмотрим простой линейный классификатор текста, который позволяет определить, к какой из трех тем относится текст (автомобили, мода, компьютеры).  
Пусть имеется словарь, содержащий набор $V$ слов естественного языка.  
Вектор $X$ - числовое представление текста, в котором каждому индексу соответствует количество вхождений слова из словаря в этом тексте.  
$W$ - матрица весов размерностью $(V, C)$, где $V$ - количество слов в словаре, $C$ - количество классов или тем.   
В результате умножения вектора $X$ на матрицу $W$ получаем вектор $Z$, в котором каждому элементу соответствует некоторое число. Чем больше число, тем выше вероятность того, что текст принадлежит классу $C$.  
Пример линейной классификации текста на три темы представлен на рисунке.    

<img src='imgs/linear_class.jpg' width=640>  



Каждая строка матрицы $W$ умножается на вектор $X$ и результат суммируется. Первая строка отвечает за первый класс, т.е. значения элементов первой строки отражают насколько каждое слово релевантно первому классу. Аналогично со второй и третьей строкой, классов может быть произвольное количество.   
Посмотрим внимательно на первую строку. Слова (авто, шина, двигатель) очень похожи на автомобильную тему, поэтому значения там положительные (не обязательно единицы, чем больше, тем выше соответствие). Слово (блок) не однозначно пренадлежит к авто-теме, поэтому там значение w будет ниже. Остальные слова однозначно не соответствуют авто-теме, поэтому там значения w меньше нуля.  
Заметим, что в нашем примере слов немного и они сгруппированы в словаре по темам. В реальных задачах размер словаря составляет несколько десятков тысяч слов, часто перемешанных в произвольном порядке. 

Числа в векторе $Z$ показывают, насколько сильно коррелирует векторное представление текста $X$ с каждой из строк матрицы $W$. Удобно перейти от этих чисел к распределению вероятностей, так чтобы каждый элемент соответствовал вероятности принадлежности текста к заданному классу, а сумма вероятностей равнялась единице.  
Для этого используют преобразование softmax(z).  

$$\sigma (z)_{i}={\frac {e^{z_{i}}}{\displaystyle \sum _{k\mathop {=} 1}^{K}e^{z_{k}}}}$$

In [7]:
import numpy as np
np.set_printoptions(precision=2)

def softmax(z):
    e = np.exp(z)
    return e / np.sum(e)

softmax([-2, 2, -4])

array([0.02, 0.98, 0.  ])

### 2) Обучение линейного классфикатора

Возникает вопрос: как найти значения коэффициентов матрицы $W$? Вручную размечать слова было бы очень утомительно. Автоматический поиск лучших коэффициентов на размеченных данных называется **обучением** классификатора.   
Для обучения нам нужно иметь достаточно большой корпус текстов (хотя бы >20), с размеченными классами.  Т.е. каждому вектору $X$ поставить в соответствие вектор $y$ например (0, 1, 0) который показывает пренадлежность ко второму классу.  
Имея для каждого $X$ предсказанное распределение вероятностей $s$, мы можем посчитать значение функции потерь (loss), обычно это кросс-энтропия.  

$${H} (s,y)=-\sum _{c}y\,\log s(x)$$ 

С учетом того, что в размеченном векторе $y$ все значения кроме правильного класса равны 0, то кросс-энтропию можно легко посчитать просто взяв логарифм от предсказанной вероятности для правильного класса.  
Если классификатор предсказывает правильному классу вероятность близкую к 1, то значение loss будет близко к нулю  
$$-log(1) = 0$$
Если классификатор предсказывает правильному классу низкую вероятность, например 0.1, то значение loss будет высоким.  
$$-log(0.1)=2.7$$
Таким образом, нам нужно найти такие значения весов матрицы $W$, при которых loss будет минимальным.  
$$loss(X,y,W) \to min $$
Эта оптимизационная задача хорошо решается методом **backpropagation** (обратное распространение ошибки), разработанным в  1974 году независимо и одновременно Галушкиным А.И. и Полом Дж. Вербосом.

Существует множество методов поиска оптимальных весов:  
- случайный перебор (плохая идея);
- стохастический градиентный спуск [SGD](https://ru.wikipedia.org/wiki/Стохастический_градиентный_спуск)  
- Momentum SGD, RMS, Adam и [другие](https://cs231n.github.io/neural-networks-3/#sgd)

### 3) Многослойный персептрон

Простой линейный классификатор подходит для несложных задач классификации. Для выделения сложных зависимостей используют комбинацию из линейных и нелинейных функций. 

<img src='imgs/multilayer.jpg'>  

Выход первого слоя - все тот же вектор $z$, является входом для следующего, нелинейного слоя.  
Нелинейные слой - это функция активации, которая позволяет лучше решать задачу классификации. На практике часто используют:  
- сигмоид;  
- гиперболический тангенс;  
- ReLU (Rectified Linear Unit) и ее модификации.

<img src='imgs/nonlinear.png' width=640>

### 4) Рекуррентные сети.

Как мы уже говорили выше, простая модель текста BagOfWords не учитывает порядок слов в предложении. Контекст очень важен для понимания смысла слов, поэтому для повышения качества классификации (и не только) нужен более продвинутый способ обработки текста. Рекуррентные нейросети - один из таких способов. 

<img src='imgs/rnn.jpg' width=640>

Каждый токен входного текста представляется в виде вектора $x_i$. Он стыкуется с некоторым начальным значением $h_0$ (обычно нулевым) и подается на вход линейного слоя $W$. Далее на нелинейный слой, опять линейный $H$ и результат в виде вектора $h_{t+1}$ стыкуется со следующим токеном $x_{t+1}$ и все повторяется по кругу. Важно понимать, что веса матриц $W$ и $H$ одинаковы для всех токенов. Обычно архитектуру RNN изображают более компактным способом.

<img src='imgs/rnn_.jpg' width=640>

Для обозначения конца петли в конце последовательности $X$ добавляют специальный токен $<END>$. Таким образом, в весах матриц $W$ и $H$ сохраняется информация о порядке расположения токенов.

Рекуррентные сети позволяют решать много других задач, например предсказание следующего слова в предложении (языковая модель). Выход $y$ в этом случае будет векторное представление предсказанного слова, той же размерности, что и $x$.  
Например:  

Было не холодно, дождь шел. Он был одет в **куртку**.  

Было холодно, дождь не шел. Он был одет в **шубу**.   

В зависимости от порядка слов, вероятность для слов куртка и шуба будет разной.  

Более продвинутой рекуррентной архитектурой является LSTM (Long Short Term Memory), которая использует отдельные веса для хранения слов, расположенных рядом и расположенных далеко друг от друга.  
Например:  

Было холодно, дождь не шел. Мне повстречался человек, которого я принял за своего знакомого. Он был одет в **шубу**.   

Реализация и обучение линейного классификатора на numpy.